In [6]:
import pandas as pd
import numpy as np
import cvxpy as cp

In [9]:
demand=pd.read_csv("DemandTotal.csv")
demand.head()

,Unnamed: 0,demand,date,time,EVdemand,demandW/OEV,demandZEV,demand1/2ZEV,demand3/4ZEV
0,0,22443,20180101,0,204.38844,21318.86358,38487.49254,29903.17806,34195.3353
1,1,23613,20180101,1,162.34800,22720.08600,36357.31800,29538.70200,32948.0100
2,2,26544,20180101,2,126.07116,25850.60862,36440.58606,31145.59734,33793.0917
3,3,26696,20180101,3,102.35652,26133.03914,34730.98682,30432.01298,32581.4999
4,4,25926,20180101,4,91.35912,25423.52484,33097.69092,29260.60788,31179.1494


In [10]:
generator=pd.read_csv("GeneratorAggregated.csv")

#DemandBase Scenario

#specify from which hour to which hour we are running this model, used for testing
i=8760
j=1

#get capacity max and min data in the right shape
capacityMax=np.repeat(generator['Nameplate.Capacity..MW.'].to_numpy().reshape(1,40),i).reshape(40,i).T
capacityMin=np.repeat(generator['Minimum.Load..MW.'].to_numpy().reshape(1,40),i).reshape(40,i).T
capacityZero=np.zeros((i,40))
#get demand data
demandBase=demand['demand'].to_numpy()[(j-1):(i+j-1)]
#get cost data
cost=generator['variableCost'].to_numpy().reshape(40,1)
#Optimization
r = cp.Variable((i,40)) # a matrix of actual capacity
# Create three constraints.
constraints = [ cp.sum(r, axis=1) == demandBase,
               r<=capacityMax,
               r>=capacityZero]
# Form objective.
obj = cp.Minimize(cp.sum(r@cost))
# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve(solver=cp.ECOS) # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", r.value)
result_demand=r.value

SyntaxError: invalid syntax (<ipython-input-10-2b8513083aaa>, line 3)

In [12]:
#demand 1/2 EV scenario


#specify from which hour to which hour we are running this model, used for testing
i=8760
j=1
#get capacity max and min data in the right shape
capacityMax=np.repeat(generator['Nameplate.Capacity..MW.'].to_numpy().reshape(1,40),i).reshape(40,i).T
capacityMin=np.repeat(generator['Minimum.Load..MW.'].to_numpy().reshape(1,40),i).reshape(40,i).T
capacityZero=np.zeros((i,40))
#get demand data
demandZEV=demand['demand1/2ZEV'].to_numpy()[(j-1):(i+j-1)]
#get cost data
cost=generator['variableCost'].to_numpy().reshape(40,1)
#Optimization
r = cp.Variable((i,40)) # a matrix of capacity
# Create constraints.
constraints = [ cp.sum(r, axis=1) == demandZEV,
               r<=capacityMax,
               r>=capacityZero]
# Form objective.
obj = cp.Minimize(cp.sum(r@cost))
# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve(solver=cp.ECOS) # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", r.value)
result_demandEVHalf=r.value

status: optimal
optimal value 581359279.3239548
optimal var [[ 2.33599999e+02  2.32300000e+03  1.05437000e+04 ...  4.27919607e-09
   3.52192637e-09 -2.98074001e-10]
 [ 2.33599999e+02  2.32300000e+03  1.05437000e+04 ...  4.27729288e-09
   3.52004527e-09 -2.99868145e-10]
 [ 2.33599999e+02  2.32300000e+03  1.05437000e+04 ...  4.28397078e-09
   3.52671029e-09 -2.93106097e-10]
 ...
 [ 2.33599999e+02  2.32300000e+03  1.05437000e+04 ...  4.40596581e-09
   3.64244788e-09 -2.05552027e-10]
 [ 2.33599999e+02  2.32300000e+03  1.05437000e+04 ...  4.29181415e-09
   3.53453120e-09 -2.84672162e-10]
 [ 2.33599999e+02  2.32300000e+03  1.05437000e+04 ...  4.28525015e-09
   3.52800243e-09 -2.91705428e-10]]


In [11]:
#demand 3/4 EV scenarios
i=8760
j=1

capacityMax=np.repeat(generator['Nameplate.Capacity..MW.'].to_numpy().reshape(1,40),i).reshape(40,i).T
capacityMin=np.repeat(generator['Minimum.Load..MW.'].to_numpy().reshape(1,40),i).reshape(40,i).T
capacityZero=np.zeros((i,40))
demandZEV2=demand['demand3/4ZEV'].to_numpy()[(j-1):(i+j-1)]
cost=generator['variableCost'].to_numpy().reshape(40,1)
#Optimization
r = cp.Variable((i,40)) # a matrix
# Create two constraints.
constraints = [ cp.sum(r, axis=1) == demandZEV2,
               r<=capacityMax,
               r>=capacityZero]
# Form objective.
obj = cp.Minimize(cp.sum(r@cost))
# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve(solver=cp.ECOS) # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", r.value)
result_demandEV75=r.value

status: optimal
optimal value 1036891852.7823719
optimal var [[ 2.33599999e+02  2.32300000e+03  1.05437000e+04 ... -7.95065747e-09
  -8.65416136e-09 -1.22200554e-08]
 [ 2.33599999e+02  2.32300000e+03  1.05437000e+04 ... -7.94753224e-09
  -8.65117873e-09 -1.22159380e-08]
 [ 2.33599999e+02  2.32300000e+03  1.05437000e+04 ... -7.94973068e-09
  -8.65327017e-09 -1.22187651e-08]
 ...
 [ 2.33600000e+02  2.32300000e+03  1.05437000e+04 ... -7.17901180e-09
  -7.92973759e-09 -1.17298767e-08]
 [ 2.33600000e+02  2.32300000e+03  1.05437000e+04 ... -7.16557795e-09
  -7.91616676e-09 -1.17106549e-08]
 [ 2.33599999e+02  2.32300000e+03  1.05437000e+04 ... -7.85547599e-09
  -8.56470485e-09 -1.21612585e-08]]


In [100]:
# NO EV scenario: create result that has MCP, system-cost, supply, demand, and customer-cost
cost_matrix = np.tile(cost.T, (8760, 1))
result=pd.DataFrame(result_demand)
result=result.round(0)
#calculate MCP
result01=(result!=0).astype(int)
result_ele = result01 * cost_matrix
result['MCP']=np.max(result_ele, axis=1)
#calculate system cost
result['system_cost']=result.loc[:,result.columns!='MCP']@cost
#calcuate others
result['supply']=np.sum(result.loc[:,result.columns!='MCP'],axis=1)-result['system_cost']
result['customer_cost']=result['MCP']*result['supply']
# show result
result

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,MCP,system_cost,supply,demand,customer_cost
0,234.0,2323.0,10544.0,9343.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.7,15883.1,22444.0,22443,38154.8
1,234.0,2323.0,10544.0,10068.0,444.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.2,18092.4,23613.0,23613,51948.6
2,234.0,2323.0,10544.0,10068.0,3375.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.2,24540.6,26544.0,26544,58396.8
3,234.0,2323.0,10544.0,10068.0,3527.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.2,24875.0,26696.0,26696,58731.2
4,234.0,2323.0,10544.0,10068.0,2757.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.2,23181.0,25926.0,25926,57037.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,234.0,2323.0,10544.0,10068.0,336.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.2,17854.8,23505.0,23505,51711.0
8756,234.0,2323.0,10544.0,9768.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.7,16605.6,22869.0,22868,38877.3
8757,234.0,2323.0,10544.0,9321.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.7,15845.7,22422.0,22421,38117.4
8758,234.0,2323.0,10544.0,9057.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.7,15396.9,22158.0,22157,37668.6


In [101]:
# 50% EV scenario: create result that has MCP, system-cost, supply, demand, and customer-cost
resultEV50=pd.DataFrame(result_demandEVHalf)
resultEV50=resultEV50.round(0)

#calculate MCP
result01=(resultEV50!=0).astype(int)
result_ele = result01 * cost_matrix
resultEV50['MCP']=np.max(result_ele, axis=1)
#calculate system cost
resultEV50['system_cost']=resultEV50.loc[:,resultEV50.columns!='MCP']@cost
#calcuate others
resultEV50['supply']=np.sum(resultEV50.loc[:,resultEV50.columns!='MCP'],axis=1)-resultEV50['system_cost']
resultEV50['customer_cost']=resultEV50['MCP']*resultEV50['supply']
# show result
resultEV50

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,MCP,system_cost,supply,demand,customer_cost
0,234.0,2323.0,10544.0,10068.0,6078.0,657.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.40,32064.0000,29904.0,29903.17806,71769.60
1,234.0,2323.0,10544.0,10068.0,6078.0,292.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.40,31188.0000,29539.0,29538.70200,70893.60
2,234.0,2323.0,10544.0,10068.0,6078.0,1899.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.40,35044.8000,31146.0,31145.59734,74750.40
3,234.0,2323.0,10544.0,10068.0,6078.0,1186.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.40,33333.6000,30433.0,30432.01298,73039.20
4,234.0,2323.0,10544.0,10068.0,6078.0,14.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.40,30520.8000,29261.0,29260.60788,70226.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,234.0,2323.0,10544.0,10068.0,6078.0,6154.0,340.0,60.0,219.0,10.0,...,0.0,0.0,0.0,0.0,0.0,5.90,88517.8337,44635.0,44635.28508,263346.50
8756,234.0,2323.0,10544.0,10068.0,6078.0,6154.0,340.0,60.0,219.0,10.0,...,0.0,0.0,0.0,0.0,0.0,5.90,60610.8337,39905.0,39905.14004,235439.50
8757,234.0,2323.0,10544.0,10068.0,6078.0,6154.0,340.0,60.0,219.0,10.0,...,0.0,0.0,0.0,0.0,0.0,2.86,47239.3200,36187.0,36186.61730,103494.82
8758,234.0,2323.0,10544.0,10068.0,6078.0,4290.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.40,40783.2000,33537.0,33536.48966,80488.80


In [102]:
# 75% EV scenario: create result that has MCP, system-cost, supply, demand, and customer-cost
resultEV75=pd.DataFrame(result_demandEV75)
resultEV75=resultEV75.round(0)

#calculate MCP
result01=(resultEV75!=0).astype(int)
result_ele = result01 * cost_matrix
resultEV75['MCP']=np.max(result_ele, axis=1)
#calculate system cost
resultEV75['system_cost']=resultEV75.loc[:,resultEV75.columns!='MCP']@cost
#calcuate others
resultEV75['supply']=np.sum(resultEV75.loc[:,resultEV75.columns!='MCP'],axis=1)-resultEV75['system_cost']
resultEV75['customer_cost']=resultEV75['MCP']*resultEV75['supply']
# show result
resultEV75


,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,MCP,system_cost,supply,demand,customer_cost
0,234.0,2323.0,10544.0,10068.0,6078.0,4949.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.40,42364.8000,34196.0,34195.3353,82070.40
1,234.0,2323.0,10544.0,10068.0,6078.0,3702.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.40,39372.0000,32949.0,32948.0100,79077.60
2,234.0,2323.0,10544.0,10068.0,6078.0,4547.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.40,41400.0000,33794.0,33793.0917,81105.60
3,234.0,2323.0,10544.0,10068.0,6078.0,3335.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.40,38491.2000,32582.0,32581.4999,78196.80
4,234.0,2323.0,10544.0,10068.0,6078.0,1933.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.40,35126.4000,31180.0,31179.1494,74832.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,234.0,2323.0,10544.0,10068.0,6078.0,6154.0,340.0,60.0,219.0,10.0,...,0.0,0.0,0.0,0.0,0.0,12.30,180060.8782,56792.0,56792.4354,698541.60
8756,234.0,2323.0,10544.0,10068.0,6078.0,6154.0,340.0,60.0,219.0,10.0,...,0.0,0.0,0.0,0.0,0.0,6.02,119046.2537,49708.0,49707.3302,299242.16
8757,234.0,2323.0,10544.0,10068.0,6078.0,6154.0,340.0,60.0,219.0,10.0,...,0.0,0.0,0.0,0.0,0.0,5.90,85396.7337,44106.0,44106.5615,260225.40
8758,234.0,2323.0,10544.0,10068.0,6078.0,6154.0,340.0,60.0,219.0,10.0,...,0.0,0.0,0.0,0.0,0.0,5.90,61661.0337,40083.0,40083.5933,236489.70


In [104]:
resultEV50=pd.DataFrame(result_demandEVHalf)
resultEV50=resultEV50.round(0)

#calculate MCP
result01=(resultEV50!=0).astype(int)
result_ele = result01 * cost_matrix
resultEV50['MCP']=np.max(result_ele, axis=1)
#calculate system cost
resultEV50['system_cost']=resultEV50.loc[:,resultEV50.columns!='MCP']@cost
#calcuate others
resultEV50['supply']=np.sum(resultEV50.loc[:,resultEV50.columns!='MCP'],axis=1)-resultEV50['system_cost']
resultEV50['demand']=demandZEV
resultEV50['customer_cost']=resultEV50['MCP']*resultEV50['supply']
# show result
resultEV50

In [106]:
result.to_csv('resultWOEV.csv')
resultEV75.to_csv('resultEV75.csv')
resultEV50.to_csv('resultEV50.csv')